In [11]:
import pandas as pd 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import gmean

In [2]:
# import data for the companies: Microsoft, Amazon, Nvidia, Tesla, Visa, Home Depot, Walmart, Netflix, Oracle, Pfizer
companies = ['MSFT','AMZN','NVDA', 'TSLA', 'V', 'HD', 'WMT', 'NFLX', 'ORCL', 'PFE']

# downloads all relevant company data to individual variables
for company in companies: 
    locals()[company] = pd.DataFrame(yf.download(tickers=company, start='2013-10-31', end ='2023-11-01', interval="1d", auto_adjust=False, prepost=False))

companiesDF = pd.DataFrame(yf.download(tickers=companies, start='2020-10-31', end ='2023-11-01', interval="1d", auto_adjust=False, prepost=False))
companiesDF = companiesDF['Adj Close']
companiesDF = (companiesDF.pct_change().dropna()*100)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  10 of 10 completed


In [3]:
# initialize a dataframe and perform preprocessing such that previous input values and the target value is contained within one row
def preprocess(ticker, returns_scaled):

    output_df = pd.DataFrame()
    index_dates = (returns_scaled.index)[5:]

    output_df['target_date'] = (index_dates)
    output_df['T-5'] = returns_scaled[ticker].values[0:(len(returns_scaled[ticker])-5)]
    output_df['T-4'] = returns_scaled[ticker].values[1:(len(returns_scaled[ticker])-4)]
    output_df['T-3'] = returns_scaled[ticker].values[2:(len(returns_scaled[ticker])-3)]
    output_df['T-2'] = returns_scaled[ticker].values[3:(len(returns_scaled[ticker])-2)]
    output_df['T-1'] = returns_scaled[ticker].values[4:(len(returns_scaled[ticker])-1)]
    output_df['T - target'] = returns_scaled[ticker].values[5:(len(returns_scaled[ticker]))]

    dates = np.array(output_df['target_date'])
    mid_cols = np.array(output_df.iloc[:, 1:6])
    y = np.array(output_df['T - target'])
    X = mid_cols.reshape(748, 5, 1)

    percentile_80 = int(len(dates) * .8)
    percentile_90 = int(len(dates) * .9)

    dates_train, X_train, y_train = dates[:percentile_80], X[:percentile_80], y[:percentile_80]
    dates_test, X_test, y_test = dates[percentile_90:], X[percentile_90:], y[percentile_90:]

    return X_train, y_train, X_test, y_test, dates_test

In [4]:
from keras.callbacks import ModelCheckpoint
import os

# creating model checkpoints for each of the metrics we want to optimize, was used to generate content in the NeuralNetowrkAnalysis file

epoch_file_path = (os.getcwd() + str('\epoch_optimization'))

epoch_checkpoint = ModelCheckpoint(
filepath=epoch_file_path,
monitor="val_loss",
mode='min',
save_best_only=True,
verbose=1
)

batch_file_path = (os.getcwd() + str('\\batch_optimization'))

batch_checkpoint = ModelCheckpoint(
    filepath=batch_file_path,
    monitor="val_loss",
    mode='min',
    save_best_only=True,
    verbose=1
) 

learning_rate_path = (os.getcwd() + str('\learning_rate_optimization'))

learning_rate_checkpoint = ModelCheckpoint(
    filepath=learning_rate_path,
    monitor="val_loss",
    mode='min',
    save_best_only=True,
    verbose=1
) 

In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
from keras.losses import MeanAbsoluteError
from keras.optimizers import Adam

model = Sequential()
model.add(Input((5, 1))) 
model.add(LSTM(96, activation='relu')) 
model.add(Dense(48, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 96)                37632     
                                                                 
 dense (Dense)               (None, 48)                4656      
                                                                 
 dense_1 (Dense)             (None, 1)                 49        
                                                                 
Total params: 42337 (165.38 KB)
Trainable params: 42337 (165.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
# see NeuralNetworkAnalysis for how I arrived at epochs and batch size selection

def test_fitting(X_train, y_train, X_test):
    model.compile(loss=MeanAbsoluteError(), optimizer=Adam(learning_rate=0.01))
    model.fit(X_train, y_train, epochs=200, validation_split=0.1, batch_size=64)
    test_predictions = model.predict(X_test).flatten()

    return test_predictions

In [7]:
reality_returns = pd.DataFrame(yf.download(tickers='AMZN', start='2023-10-31', interval="1d", auto_adjust=False, prepost=False))
reality_returns = reality_returns.pct_change().dropna()*100
reality_returns_dates = reality_returns.index
reality_returns_open = reality_returns['Open']

[*********************100%***********************]  1 of 1 completed


In [8]:
# Future predictions

def predict_future(X_test, test_dates, test_predictions, model):

    recent_data = (X_test[-1]) #X test
    temp = (test_predictions[-1])
    future_predictions = []
    future_days = 28
    future_dates = []

    for increment in range(1, future_days+1):
        next_day = np.array(test_dates[-1] + np.timedelta64(increment, 'D'))
        future_dates.append(next_day)

    n = 0

    while n < future_days:
        new_array = np.delete(recent_data, 0)
        new_array = np.append(new_array, temp)
        recent_data = np.array(new_array).reshape(1, 5, 1)
        temp = model.predict(recent_data)
        future_predictions.append(temp)
        n += 1

    return np.asarray(future_predictions).reshape(future_days)

In [9]:
results = preprocess('MSFT', pd.DataFrame(companiesDF['MSFT']))

In [10]:
results = {}

for ticker in companies:
    display(ticker)
    returns_scaled = pd.DataFrame(companiesDF[ticker])
    X_train = preprocess(ticker, returns_scaled)[0] 
    y_train = preprocess(ticker, returns_scaled)[1] 
    X_test = preprocess(ticker, returns_scaled)[2] 
    y_test = preprocess(ticker, returns_scaled)[3] 
    test_dates = preprocess(ticker, returns_scaled)[4]
    test_predictions = test_fitting(X_train, y_train, X_test)
    daily_returns = predict_future(X_test, test_dates, test_predictions, model)
    results[ticker] = daily_returns

'MSFT'

Epoch 1/200
9/9 [==============================] - 1s 28ms/step - loss: 1.3491 - val_loss: 1.5254
Epoch 2/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3398 - val_loss: 1.5349
Epoch 3/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3331 - val_loss: 1.5335
Epoch 4/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3295 - val_loss: 1.5327
Epoch 5/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3249 - val_loss: 1.5403
Epoch 6/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3119 - val_loss: 1.5228
Epoch 7/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3096 - val_loss: 1.5899
Epoch 8/200
9/9 [==============================] - 0s 11ms/step - loss: 1.3290 - val_loss: 1.5278
Epoch 9/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3263 - val_loss: 1.5235
Epoch 10/200
9/9 [==============================] - 0s 6ms/step - loss: 1.3205 - val_loss: 1.5308
Epoch 11/200
9/9 [=========

'AMZN'

Epoch 1/200
9/9 [==============================] - 1s 25ms/step - loss: 1.9298 - val_loss: 2.0168
Epoch 2/200
9/9 [==============================] - 0s 6ms/step - loss: 1.6511 - val_loss: 1.9520
Epoch 3/200
9/9 [==============================] - 0s 6ms/step - loss: 1.5829 - val_loss: 1.9468
Epoch 4/200
9/9 [==============================] - 0s 7ms/step - loss: 1.5205 - val_loss: 1.9836
Epoch 5/200
9/9 [==============================] - 0s 7ms/step - loss: 1.4512 - val_loss: 2.0284
Epoch 6/200
9/9 [==============================] - 0s 8ms/step - loss: 1.3758 - val_loss: 2.0823
Epoch 7/200
9/9 [==============================] - 0s 7ms/step - loss: 1.3205 - val_loss: 2.1095
Epoch 8/200
9/9 [==============================] - 0s 7ms/step - loss: 1.2742 - val_loss: 2.1710
Epoch 9/200
9/9 [==============================] - 0s 7ms/step - loss: 1.2256 - val_loss: 2.2469
Epoch 10/200
9/9 [==============================] - 0s 7ms/step - loss: 1.2176 - val_loss: 2.1967
Epoch 11/200
9/9 [==========

'NVDA'

...

Epoch 1/200
9/9 [==============================] - 2s 38ms/step - loss: 2.3031 - val_loss: 1.9004
Epoch 2/200
9/9 [==============================] - 0s 8ms/step - loss: 2.0451 - val_loss: 1.8351
Epoch 3/200
9/9 [==============================] - 0s 9ms/step - loss: 1.8897 - val_loss: 1.8271
Epoch 4/200
9/9 [==============================] - 0s 9ms/step - loss: 1.7475 - val_loss: 1.8588
Epoch 5/200
9/9 [==============================] - 0s 9ms/step - loss: 1.6626 - val_loss: 1.9078
Epoch 6/200
9/9 [==============================] - 0s 10ms/step - loss: 1.6397 - val_loss: 1.9359
Epoch 7/200
9/9 [==============================] - 0s 9ms/step - loss: 1.5493 - val_loss: 1.9697
Epoch 8/200
9/9 [==============================] - 0s 10ms/step - loss: 1.5088 - val_loss: 2.0250
Epoch 9/200
9/9 [==============================] - 0s 9ms/step - loss: 1.3820 - val_loss: 2.0594
Epoch 10/200
9/9 [==============================] - 0s 16ms/step - loss: 1.3411 - val_loss: 2.0854
Epoch 11/200
9/9 [=======

'ORCL'

Epoch 1/200
9/9 [==============================] - 2s 40ms/step - loss: 1.5730 - val_loss: 1.1954
Epoch 2/200
9/9 [==============================] - 0s 9ms/step - loss: 1.3130 - val_loss: 1.1293
Epoch 3/200
9/9 [==============================] - 0s 8ms/step - loss: 1.1521 - val_loss: 1.1035
Epoch 4/200
9/9 [==============================] - 0s 8ms/step - loss: 1.0669 - val_loss: 1.1059
Epoch 5/200
9/9 [==============================] - 0s 9ms/step - loss: 0.9852 - val_loss: 1.1315
Epoch 6/200
9/9 [==============================] - 0s 10ms/step - loss: 0.9255 - val_loss: 1.1631
Epoch 7/200
9/9 [==============================] - 0s 9ms/step - loss: 0.8703 - val_loss: 1.1438
Epoch 8/200
9/9 [==============================] - 0s 9ms/step - loss: 0.8079 - val_loss: 1.1772
Epoch 9/200
9/9 [==============================] - 0s 9ms/step - loss: 0.7616 - val_loss: 1.1805
Epoch 10/200
9/9 [==============================] - 0s 10ms/step - loss: 0.7379 - val_loss: 1.1997
Epoch 11/200
9/9 [========

'PFE'

Epoch 1/200
9/9 [==============================] - 2s 42ms/step - loss: 1.4878 - val_loss: 1.1372
Epoch 2/200
9/9 [==============================] - 0s 8ms/step - loss: 1.2224 - val_loss: 1.0701
Epoch 3/200
9/9 [==============================] - 0s 9ms/step - loss: 1.1084 - val_loss: 1.0496
Epoch 4/200
9/9 [==============================] - 0s 9ms/step - loss: 1.0032 - val_loss: 1.0633
Epoch 5/200
9/9 [==============================] - 0s 8ms/step - loss: 0.9228 - val_loss: 1.0889
Epoch 6/200
9/9 [==============================] - 0s 9ms/step - loss: 0.8731 - val_loss: 1.0989
Epoch 7/200
9/9 [==============================] - 0s 8ms/step - loss: 0.8033 - val_loss: 1.1089
Epoch 8/200
9/9 [==============================] - 0s 9ms/step - loss: 0.7396 - val_loss: 1.0703
Epoch 9/200
9/9 [==============================] - 0s 8ms/step - loss: 0.7043 - val_loss: 1.0930
Epoch 10/200
9/9 [==============================] - 0s 8ms/step - loss: 0.6438 - val_loss: 1.0900
Epoch 11/200
9/9 [==========

In [1]:
#fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(25,10))


In [181]:
# daily returns over the next 10 days that markets are open

geomean_results = {}

def return_geomean(results):
    for key in results.keys():
        geomean = gmean(100+results[key])-100
        geomean_results[key] = geomean

    return geomean_results

display(return_geomean(results))

{'MSFT': 0.0817108154296875,
 'AMZN': 0.00319671630859375,
 'NVDA': 0.5059661865234375,
 'TSLA': 0.8804855346679688,
 'V': -0.2744140625,
 'HD': 0.43085479736328125,
 'WMT': -0.2091522216796875,
 'NFLX': -0.07778167724609375,
 'ORCL': 0.09384918212890625,
 'PFE': 0.20793914794921875}